In [ ]:
import cv2
import os
import numpy as np
import shutil
import torch
import zipfile
import cv2
import os
import numpy as np
import pandas as pd
import datetime as dt
dataset = pd.read_csv('dataset.csv')

In [ ]:
import spacy
import re
import string
tok = spacy.load('en')
def tokenize (text):
    text = re.sub(r"[^\x00-\x7F]+", " ", text)
    text = no_specials_string = re.sub('[!#?,.:";]', ' ', text)
    regex = re.compile('[' + re.escape(string.punctuation) + '\\r\\t\\n]')
    nopunct = regex.sub(" ", text.lower())
    tmp = [token.text for token in tok.tokenizer(nopunct)]
    return [x for x in tmp if not x.isspace()]

In [ ]:
for i in range(len(dataset)):
    dataset['video_title'][i] = tokenize(dataset['video_title'][i])

In [ ]:
vocab2id = {"":0, "UNK":1}
words = ["", "UNK"]

from collections import Counter
counts_words = Counter()
for row in dataset['video_title']:
    counts_words.update(row)
for word in counts_words:
    vocab2id[word] = len(words)
    words.append(word)
encoded_texts = []
dataset['num_words'] = 0
for i in range(len(dataset)):
    dataset['num_words'][i] = len(dataset['video_title'][i])

In [ ]:
dataset = dataset[dataset['num_words'] != 0]
dataset = dataset.reset_index(drop = True)
max_len_title = max(dataset['num_words'])

In [ ]:
for elem in dataset['video_title']:
    enc = np.zeros(max_len_title)
    smth = np.array([vocab2id.get(word, vocab2id["UNK"]) for word in elem])
    enc[:len(smth)] = smth
    encoded_texts.append(enc)

In [ ]:
encoded_texts = np.array(encoded_texts,  dtype =  int)

In [ ]:
tmp = pd.Series( (v for v in encoded_texts) )

In [ ]:
dataset['Encoded'] = tmp

In [ ]:
word_vectors = {}
with open("glove/glove.twitter.27B.100d.txt") as f:
    for line in f:
        arr = line.split()
        word_vectors[arr[0]] = np.array([float(x) for x in arr[1:]])

In [ ]:
vocab_size = len(counts_words) + 2
vocab_to_idx = {}
vocab = ["", "UNK"]
W = np.zeros((vocab_size, 100), dtype="float32")
W[0] = np.zeros(100, dtype='float32')
W[1] = np.random.uniform(-0.25, 0.25, 100)
vocab_to_idx["UNK"] = 1
i = 2
for word in counts_words:
    if word in word_vectors:
        W[i] = word_vectors[word]
    else:
        W[i] = np.random.uniform(-0.25,0.25, 100)
    vocab_to_idx[word] = i
    vocab.append(word)
    i += 1   

In [ ]:
from torch.utils import data
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
class text_dataset(data.Dataset):
    def __init__(self, text, y):
        "Initialization"
        self.text = text
        self.y = y
    
    def __len__(self):
        return len(self.y)
    
    def read_images_folder(self, folder):
        frames_vid = os.listdir(os.path.join(self.data_path_frame, folder))
        def myFunc(e):
            return int(e.split('.')[0])
        out = []
        frames_vid.sort(reverse=False, key=myFunc)
        for pic in frames_vid:
            image = Image.open(os.path.join(self.data_path_frame, folder, pic))
            if self.transform_frames:
                image = self.transform_frames(image)
            out.append(image)
        out = torch.stack(out, dim = 0)
        return out
    
    def __getitem__(self, index):
        encoded_text = torch.from_numpy(self.text[index][0])
        length_text = self.text[index][1]
        y = list(self.y)[index]
        return (encoded_text, length_text), y

In [ ]:
from torch.nn.utils.rnn import pack_padded_sequence
class BiLSTM(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers,  glove_weights):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.embeddings.weight.data.copy_(torch.from_numpy(glove_weights))
        self.embeddings.weight.requires_grad = False
        self.lstm = nn.LSTM(input_size = embedding_dim,
                            hidden_size = 128,
                            bidirectional = True,
                            num_layers = num_layers,
                            dropout = 0.4,
                            batch_first=True)
        self.fc1 = nn.Linear(256, 128)
        self.func1 = nn.LeakyReLU(0.1)
        self.dropout = nn.Dropout(0.4)
        self.fc2 = nn.Linear(128, 1)
        self.sig = nn.Sigmoid()

    def forward(self, x, l):
        x = self.embeddings(x)
        x_pack = pack_padded_sequence(x, l, batch_first=True, enforce_sorted=False)
        out_st, (ht, ct) = self.lstm(x_pack)
        ht = torch.cat((ht[-2,:,:], ht[-1,:,:]),  dim = 1)
        out = self.fc1(ht)
        out = self.func1(out)
        out = self.dropout(out)
        out = self.fc2(out)
        out = self.sig(out)
        return out

In [ ]:
dataset['popular'] = 0.0
dataset['popular'][dataset.view_count_new.gt(dataset.view_count_new.quantile(0.8))] = 1.0

In [ ]:
dataset = dataset.sort_values(by = 'published_at',   ascending=False)
dataset = dataset.reset_index(drop = True)

In [ ]:
train_ids = int(0.8*len(dataset))
val_ids = int(0.8*len(dataset)) + int(0.1*len(dataset))

In [ ]:
dataset['popular'] = 0.0
dataset['popular'][dataset.view_count_new.gt(dataset.view_count_new.quantile(0.8))] = 1.0

In [ ]:
train = dataset[:train_ids]
val = dataset[train_ids:val_ids]
test = dataset[val_ids:len(dataset)]

In [ ]:
train.columns

In [ ]:
text_train = train[['Encoded', 'num_words']].values
y_train = train['popular'].values

text_val = val[['Encoded', 'num_words']].values
y_val = val['popular'].values

text_test = test[['Encoded', 'num_words']].values
y_test = test['popular'].values


In [ ]:
dataset_train = text_dataset(text_train,  y_train)
dataset_val = text_dataset(text_val, y_val)
dataset_test = text_dataset(text_test, y_test)

In [ ]:
mod = BiLSTM(vocab_size, 100, 128, 2, W ).cuda()
model = nn.DataParallel(mod)

In [ ]:
train_dl = data.DataLoader(dataset_train, batch_size = 200)
val_dl = data.DataLoader(dataset_val, batch_size = 200)

In [ ]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(y_pred)
    y_pred_tag = y_pred_tag.squeeze(dim = -1)
    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = acc.item()
    return acc

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from torch.utils.tensorboard import SummaryWriter
train_log = []
val_log = []
import matplotlib.pyplot as plt
def train_model(model, epoch, lr):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    opt_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.5)
    criterion = nn.BCELoss()
    writer = SummaryWriter()
    for i in range(epoch):
        model.train()
        sum_loss = 0.0
        total = 0
        train_log_epoch = []
        train_val_epoch = []
        num_batch = 0
        total_predicts = []
        total_corrects = []
        for elem in train_dl:
            optimizer.zero_grad()
            text = elem[0][0].cuda()
            length = elem[0][1]
            y = elem[1].float().cuda()
            y_pred = model(text, length)
            print(y.shape)
            loss = criterion(y_pred, y.unsqueeze(-1))
            loss.backward()
            optimizer.step()
            loss = loss.item()
            acc = binary_acc(y_pred, y)
            train_log_epoch.append(acc)
            y_pred_sig = y_pred.flatten().tolist()
            total_predicts.extend(y_pred_sig)
            total_corrects.extend(y.flatten().tolist())
        
        train_log.extend(train_log_epoch)
        acc = np.mean(train_log_epoch)
        binary_preds = np.array(total_predicts)>0.5
        binary_preds = binary_preds.astype(int)
        binary_preds = list(binary_preds)
        print('train')
        print(len(binary_preds))
        print(binary_preds)
        print(len(total_corrects))
        prec = precision_score(total_corrects, binary_preds)
        rec = recall_score(total_corrects, binary_preds)
        roc_auc = roc_auc_score(total_corrects, total_predicts)
        writer.add_scalar('precision_train', prec, i)
        writer.add_scalar('recall_train', rec, i)
        writer.add_scalar('roc_auc_train', roc_auc, i)
        writer.add_scalar('accuracy_train', np.mean(train_log_epoch), i)
        prec_test, rec_test, roc_auc_test, acc_test = val(model, val_dl)
        opt_scheduler.step(acc_test)
        writer.add_scalar('precision_test', prec_test, i)
        writer.add_scalar('recall_test', rec_test, i)
        writer.add_scalar('roc_auc_test', roc_auc_test, i)
        writer.add_scalar('accuracy_test', acc_test, i)
        writer.close()
    return model


def val(model, valid_dl):
    model.eval()
    sum_loss = 0.0
    val_epoch = []
    total_preds = []
    total_corrects = []
    criterion = nn.BCEWithLogitsLoss()
    for elem in valid_dl:
        text = elem[0][0].cuda()
        length = elem[0][1]
        y = elem[1].float().cuda()
        y_pred = model(text, length)
        y_pred_sig = y_pred.flatten().tolist()
        total_preds.extend(y_pred_sig)
        total_corrects.extend(y.flatten().tolist())
        acc = binary_acc(y_pred, y)
        val_epoch.append(acc)
    
    binary_preds = np.array(total_preds)>0.5
    binary_preds = binary_preds.astype(int)
    binary_preds = list(binary_preds)
    print('test')
    print(total_corrects)
    print(binary_preds)
    print(total_preds)
    prec = precision_score(total_corrects, binary_preds)
    rec = recall_score(total_corrects, binary_preds)
    roc_auc = roc_auc_score(total_corrects, total_preds)
    acc = np.mean(val_epoch)
    return prec, rec, roc_auc, acc

In [ ]:
model = train_model(model, 3, 0.01)